# Detecting Fraud in Ad Impression Data

I have a file with one day of impression logs for selected IP addresses. The nine fields
are, in order:

* Timestamp
* IP address
* Detected browser type
* User agent string
* Host (URL)
* Whether the impression was in view (1.0 = yes, 0.0 = no)
* Number of plugins installed
* Browser window position and size (x, y, width, height)
* Network latency

My task is to identify hosts which are receiving a substantial amount of fraudulent
traffic. As part of this, I may also wish to identify IP addresses home to machines that
are part of botnets, but this is not required. The definition of "substantial" is up to me --
this may be a ranked list of all hosts, or a list of hosts reaching a certain threshold, or I
may choose not to quantify the amount of fraud and simply classify hosts as likely to be
experiencing high fraud or not. To get started, I have a list of hosts which are known
to receive substantial amounts of fraudulent traffic:

* featureball.com
* uvido.com
* sprungliving.com
* sweetboxgames.com
* mammabay.co.uk
* workingfathertv.com
* worsthorrorgame.com
* hourlyparent.com
* ulterior-movies.com
* myhomesdesign.com
* indoorlife.tv
* bumclub.info
* psychoworld.tv
* hunp.us
* rlinevideos.com


In [33]:
import re
import math
import numpy as np
print 'numpy', np.__version__
import pandas as pd
print 'pandas', pd.__version__
import seaborn as sns
print 'seaborn', sns.__version__
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

pd.options.display.max_columns = 25

imp_df = pd.io.parsers.read_csv("ad_impression_data_set.tsv",sep='\t',
                                names=['Timestamp','IPadd','Browser','UserAgent','Host','Inview','Nplugins','Wpossize','Latency'],
                               header=None)
imp_df.head(5)

numpy 1.10.1
pandas 0.17.0
seaborn 0.6.0


,Timestamp,IPadd,Browser,UserAgent,Host,Inview,Nplugins,Wpossize,Latency
0,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0,NaN,"(0,0,1280,629)",0
1,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0,NaN,"(0,0,1280,629)",0
2,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN
3,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN
4,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN


In [34]:
print imp_df.describe()

              Inview       Nplugins         Latency
count  211811.000000  162304.000000   200283.000000
mean        0.431998      10.139442      345.200182
std         0.495355       7.637172    13292.816593
min         0.000000       1.000000        0.000000
25%         0.000000       2.000000        0.000000
50%         0.000000      10.000000       54.000000
75%         1.000000      15.000000      179.000000
max         1.000000      61.000000  5866142.000000


In [37]:

def clean_hosts(data=None):
    ''' A bunch of regular expression magic to filter host names to the top level'''
    if data is None:
		raise ValueError("Input 'data' to clean_data is None")
    # deal with co.uk and com.au, convert to .couk and .comau, respectively (and other variants)
    data["Host"]=data["Host"].apply(lambda x: re.sub('(\.co)\.([a-z][a-z])$',r'\1\2',x))
    data["Host"]=data["Host"].apply(lambda x: re.sub('(\.com)\.([a-z][a-z])$',r'\1\2',x))
    # select just the top level and suffix
    data["Host"]=data["Host"].apply(lambda x: '.'.join(str(x).split('.')[-2:]))
    # account for cases like http://mlb.com (where there is only one '.')
    data["Host"]=data["Host"].apply(lambda x: str(x).split('//')[-1])
    
    #Remove ERROR Hosts
    print '\nNumber of ERROR hosts to be removed:', sum(data["Host"]=="ERROR"), '\n'
    data=data[data["Host"]!="ERROR"]
    return data

print imp_df.info()
print imp_df['Host'].head(5)
imp_df = clean_hosts(imp_df)
print imp_df.info()
print imp_df['Host'].head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234945 entries, 0 to 235082
Data columns (total 9 columns):
Timestamp    234945 non-null object
IPadd        234945 non-null object
Browser      234945 non-null object
UserAgent    234945 non-null object
Host         234945 non-null object
Inview       211810 non-null float64
Nplugins     162303 non-null float64
Wpossize     118450 non-null object
Latency      200279 non-null float64
dtypes: float64(3), object(6)
memory usage: 17.9+ MB
None
0       domain.comau
1       domain.comau
2    mangareader.net
3    mangareader.net
4    mangareader.net
Name: Host, dtype: object

Number of ERROR hosts to be removed: 0 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234945 entries, 0 to 235082
Data columns (total 9 columns):
Timestamp    234945 non-null object
IPadd        234945 non-null object
Browser      234945 non-null object
UserAgent    234945 non-null object
Host         234945 non-null object
Inview       211810 non-null float64
Nplug

In [40]:
fraudtraffic=['featureball.com',
'uvido.com',
'sprungliving.com',
'sweetboxgames.com',
'mammabay.couk', #had to modify because of how I cleaned above
'workingfathertv.com',
'worsthorrorgame.com',
'hourlyparent.com',
'ulterior-movies.com',
'myhomesdesign.com',
'indoorlife.tv',
'bumclub.info',
'psychoworld.tv',
'hunp.us',
'rlinevideos.com']

print "Host counts:\n", imp_df[imp_df["Host"].isin(fraudtraffic)]["Host"].value_counts()#.describe()

Host counts:
uvido.com              692
featureball.com        560
sprungliving.com       228
mammabay.couk          123
workingfathertv.com    120
worsthorrorgame.com     94
hourlyparent.com        85
ulterior-movies.com     83
myhomesdesign.com       69
indoorlife.tv           60
bumclub.info            59
hunp.us                 58
psychoworld.tv          57
rlinevideos.com         55
Name: Host, dtype: int64


Yikes, we're missing one host, 'sweetboxgames.com'. By searching through the file, it appears there was a typo and the host name should be 'sweetxboxgames.com' (the 'x' was missing after 'sweet'). I will modify my `fraudtraffic` list to remedy this.

In [41]:
fraudtraffic=['featureball.com',
'uvido.com',
'sprungliving.com',
'sweetxboxgames.com', #fixed typo
'mammabay.couk', #had to modify because of how I cleaned above
'workingfathertv.com',
'worsthorrorgame.com',
'hourlyparent.com',
'ulterior-movies.com',
'myhomesdesign.com',
'indoorlife.tv',
'bumclub.info',
'psychoworld.tv',
'hunp.us',
'rlinevideos.com']

print "Host counts:\n", imp_df[imp_df["Host"].isin(fraudtraffic)]["Host"].value_counts()#.describe()

Host counts:
uvido.com              692
featureball.com        560
sprungliving.com       228
sweetxboxgames.com     150
mammabay.couk          123
workingfathertv.com    120
worsthorrorgame.com     94
hourlyparent.com        85
ulterior-movies.com     83
myhomesdesign.com       69
indoorlife.tv           60
bumclub.info            59
hunp.us                 58
psychoworld.tv          57
rlinevideos.com         55
Name: Host, dtype: int64


In [55]:
print imp_df.Browser.value_counts()
print imp_df.Browser.head(5)
print 'Chrome', imp_df.UserAgent.iloc[5]
print 'Safari', imp_df.UserAgent.iloc[0]
print 'IE', imp_df.UserAgent.iloc[20]
print 'Firefox', imp_df.UserAgent.iloc[38]

Chrome               76595
Internet Explorer    63874
Safari/Webkit        44093
Firefox              28697
Unknown              21599
Opera                   87
Name: Browser, dtype: int64
0    Safari/Webkit
1    Safari/Webkit
2          Unknown
3          Unknown
4          Unknown
Name: Browser, dtype: object
Chrome Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36
Safari Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/534.59.10 (KHTML, like Gecko) Version/5.1.9 Safari/534.59.10
IE Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko
Firefox Mozilla/5.0 (Windows NT 6.1; rv:32.0) Gecko/20100101 Firefox/32.0


In [ ]:
import requests
#ua_string = imp_df.UserAgent.iloc[20]

def lookup_user_agent(ua_string):
    r = requests.post("http://www.useragentstring.com", data = {"uas":ua_string,"getJSON":'agent_name-agent_version'})
    return r.json()['agent_name']+'_'+r.json()['agent_version']

#print r.json()
#print r.json()['agent_name']
#print r.json()['agent_version']

#test_data = imp_df.iloc[0:20]
imp_df['UA_Browser_Ver']=imp_df["UserAgent"].apply(lambda x: lookup_user_agent(x))
#response = urllib2.urlopen(url).read()
#print response
#dat = simplejson.load(r.json())   
#print dat
#http://www.useragentstring.com/?uas=Mozilla/5.0%20%28Windows%20NT%206.1;%20WOW64;%20Trident/7.0;%20rv:11.0%29%20like%20Gecko&getJSON=agent_name-agent_version

In [68]:
test_data

,Timestamp,IPadd,Browser,UserAgent,Host,Inview,Nplugins,Wpossize,Latency,UA_Browser_Ver
0,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,domain.comau,0,NaN,"(0,0,1280,629)",0,Safari_5.1.9
1,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,domain.comau,0,NaN,"(0,0,1280,629)",0,Safari_5.1.9
2,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,Chrome_36.0.1985.143
3,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,Chrome_36.0.1985.143
4,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,Chrome_36.0.1985.143
5,2014-08-25 00:00:00,326.432.563.561,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,cheezburger.com,0,25,"(0,0,1366,643)",73,Chrome_36.0.1985.143
6,2014-08-25 00:00:02,445.435.337.514,Unknown,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,cars-expert.com,NaN,NaN,NaN,NaN,Internet Explorer_9.0
7,2014-08-25 00:00:03,386.335.512.400,Safari/Webkit,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,lovepoemsandquotes.com,0,1,"(0,0,1000,1163)",0,Safari_7.0
8,2014-08-25 00:00:03,488.432.432.344,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,bossip.com,1,23,NaN,386,Chrome_36.0.1985.143
9,2014-08-25 00:00:03,412.428.488.484,Chrome,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4)...,adsafeprotected.com,0,12,NaN,0,Chrome_36.0.1985.143


### Assumptions
* logs are correct. I assume that when there are observations of one IP address clicking on the same host 3 times in a second, that this is what occurred. It may be the case that some entries are simply repeated due to errors in the recording.
* ERROR hosts are to be disregarded. As there is no valid host, the only information is IP address and Browser/User Agent info
* That greatboxgames.com should be greatxboxgames.com

### Cleaning and Feature Engineering
* Remove ERROR hosts
* Standardize hosts by removing preceeding 'http://www.|go.|video.'
* Convert timestamp to seconds in the day
* Create browser window area feature
* Create inter-impression interval (III) feature

In [ ]:
def clean_data(data=None):
    if data is None:
		raise ValueError("Input 'data' to clean_data is None")

    #Clean up some of the Host strings
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://go.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://video.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://videos.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://videos1.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://vids.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://pixel.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://fw.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://ad.",""))
    
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://www.",""))
    data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://",""))
    
    #Remove ERROR Hosts
    print 'Number of ERROR hosts:', sum(data["Host"]=="ERROR")
    data=data[data["Host"]!="ERROR"]
    return data
   
def timestamp_to_secs(x):
    #Converts timestamp to seconds in the day
    return int(x[11:13])*3600  + int(x[-5:-3])*60 + int(x[-2:])

def window_area(x):
    #Multiplies the width and height of the window
    val=np.NAN
    if isinstance(x,str):
        [xp,yp,w,h]=x[1:-1].split(',')
        val=int(w)*int(h)
    return val

In [2]:
def timestamp_to_secs(x):
    return int(x[11:13])*3600  + int(x[-5:-3])*60 + int(x[-2:])
      
#Add column that converts timestamp to time of day, in seconds
data["Tsecs"]=data["Timestamp"].apply(lambda x: timestamp_to_secs(x))

#Clean up some of the Host strings
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://go.",""))
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://video.",""))
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://videos.",""))
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://vids.",""))
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://www.",""))
data["Host"] = data["Host"].apply(lambda x: str(x).replace("http://",""))

#Remove ERROR Hosts
print sum(data["Host"]=="ERROR")
data=data[data["Host"]!="ERROR"]
print sum(data["Host"]=="ERROR")

#Multiplies the width and height of the window
def window_area(x):
    val=np.NAN
    if isinstance(x,str):
        [xp,yp,w,h]=x[1:-1].split(',')
        val=int(w)*int(h)
    return val

#Add column that contains browser window area
data["Bwinarea"] = data["Bwinpossize"].apply(lambda x: window_area(x))     

138
0


In [ ]:
fraudtraffic=["featureplay.com","uvidi.com","spryliving.com","greatxboxgames.com",
              "mmabay.co.uk","workingmothertv.com","besthorrorgame.com","dailyparent.com","superior-movies.com",
              "yourhousedesign.com","outdoorlife.tv","drumclub.info","cycleworld.tv","hmnp.us","nlinevideos.com"]

print "IP counts:\n", data[data["Host"].isin(fraudtraffic)]["IPadd"].value_counts().describe()
print "Host counts:\n", data[data["Host"].isin(fraudtraffic)]["Host"].value_counts().describe()

fdata = data[data["Host"].isin(fraudtraffic)]
nfdata = data[~data["Host"].isin(fraudtraffic)]

f, axs = plt.subplots(1, 2)
fdata["Tsecs"].value_counts(normalize=True).plot(kind='bar',ax=axs[0])
axs[0].set_title('Seconds (fraud)')
nfdata["Tsecs"].value_counts(normalize=True).plot(kind='bar',ax=axs[1])
axs[1].set_title('Seconds (nonfraud)')


In [82]:

#Reduce the data - Remove Hosts that are present less than 8 times
host_counts = data["Host"].value_counts()
host_counts
host_counts.describe()
sighost_counts = host_counts[host_counts>8]
keepHosts = sighost_counts.index.map(lambda x: str(x))
data = data[data["Host"].isin(keepHosts)]


#Reduce the data - Remove IPs that are present less than 17 times
IP_counts = data["IPadd"].value_counts()
IP_counts
IP_counts.describe()
sigIP_counts = IP_counts[IP_counts>17]
keepIPs = sigIP_counts.index.map(lambda x: str(x))
data = data[data["IPadd"].isin(keepIPs)]



805120


C:\Users\btq\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [83]:
data

,Timestamp,IPadd,Browser,UserA,Host,Iinview,Nplugins,Bwinpossize,NetLat,Tsecs,Bwinarea
0,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,domain.com.au,0,NaN,"(0,0,1280,629)",0,0,805120
1,2014-08-25 00:00:00,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,domain.com.au,0,NaN,"(0,0,1280,629)",0,0,805120
2,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,0,NaN
3,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,0,NaN
4,2014-08-25 00:00:00,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,mangareader.net,NaN,NaN,NaN,NaN,0,NaN
5,2014-08-25 00:00:00,326.432.563.561,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,failblog.cheezburger.com,0,25,"(0,0,1366,643)",73,0,878338
6,2014-08-25 00:00:02,445.435.337.514,Unknown,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,cars-expert.com,NaN,NaN,NaN,NaN,2,NaN
7,2014-08-25 00:00:03,386.335.512.400,Safari/Webkit,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,lovepoemsandquotes.com,0,1,"(0,0,1000,1163)",0,3,1163000
8,2014-08-25 00:00:03,488.432.432.344,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,ads.bossip.com,1,23,NaN,386,3,NaN
9,2014-08-25 00:00:03,412.428.488.484,Chrome,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4)...,fw.adsafeprotected.com,0,12,NaN,0,3,NaN
